In [57]:
import ipywidgets as widgets
import polars as pl
import trueskill

In [58]:
season_start = '2024-11-03'
season_end = '2025-04-19'

In [59]:
candidates = pl.DataFrame([
    ["/league/nemelee/player/97D40D09-E9DF-468E-B90B-DD42CA8FA878?", "Younger"],
    ["/league/nemelee/player/583438BD-F9A1-413A-8F77-DEF3820143F0?", "N0va | hc | Coolslice"],
    ["/league/nemelee/player/BAB54F38-444F-4ADE-8D79-EC7D232D90E9?", "OUG | Electroman"],
    ["/league/nemelee/player/1D8B9E33-86AE-45FB-89E9-0CB158C24FD3?", "regEx"],
    ["/league/nemelee/player/39E544D6-60EC-4243-829E-C0E017FD58DC?", "s6 | shmeeli"],
    ["/league/nemelee/player/0267F6B4-223B-4D0D-B0AB-3B7084B37AB7?", "Greenstach"],
    ["/league/nemelee/player/EAE5E0CC-99FF-49EB-8424-142348454761?", "Comcast | The CoveCare Scare"],
    ["/league/nemelee/player/E85EE0F4-C335-4A1C-A569-BE69C1675BAA?", "$G|MP | Bank"],
    ["/league/nemelee/player/2407F41F-64A0-43AF-85F1-96F6985E57EE?", "hc | saucymain"],
    ["/league/nemelee/player/5CD3DDFC-6380-4395-8DF7-590D498437E2?", "Future Shock"],
    ["/league/nemelee/player/D3B00EC5-2479-4E29-8FBE-32DEEFFD23D2?", "Nage"],
    ["/league/nemelee/player/5DBCC342-92B6-40C6-AFBF-66933EF41379?", "Ant"],
    ["/league/nemelee/player/11CA20C1-5FDF-4A64-BC6C-0B5B91497FE7?", "MATE | 22K"],
    ["/league/nemelee/player/FA73F3AA-3D89-4584-A3D3-3E44E4B61180?", "Rat Rattington"],
    ["/league/nemelee/player/17A43CB7-E905-4A9A-A69A-55CC66EE929E?", "Bin/BTR | Yousef"],
    ["/league/nemelee/player/0CDB11B1-BCC3-4027-90F2-D74F1A9A5668?", "hc | kraft"],
], orient="row", schema=["url", "tag"])

In [60]:
candidate_urls = candidates.select('url').to_series().to_list()


# Load Data

In [61]:
matches = pl.read_csv("../../../data/matches.csv")
season_matches = (
    matches
    .filter(pl.col('tournament_date') >= season_start)
    .filter(pl.col('tournament_date') <= season_end)
)

In [72]:
players = pl.read_csv("../../../data/players.csv")

In [73]:
players_dict = {}

def rate_players(row: dict) -> dict:
    # Create TrueSkill rating objects for the players
    if players_dict.get(row['winner_url']) is None:
        players_dict[row['winner_url']] = trueskill.Rating()
    if players_dict.get(row['loser_url']) is None:
        players_dict[row['loser_url']] = trueskill.Rating()

    result = {
        'winner_rating': players_dict[row['winner_url']],
        'loser_rating': players_dict[row['loser_url']],
    }

    # Update the ratings based on the match outcome
    new_winner_rating, new_loser_rating = trueskill.rate_1vs1(
        players_dict[row['winner_url']], players_dict[row['loser_url']])
    players_dict[row['winner_url']] = new_winner_rating
    players_dict[row['loser_url']] = new_loser_rating

    result.update({
        'new_winner_rating': new_winner_rating,
        'new_loser_rating': new_loser_rating,
    })
    return result

In [74]:
players_dict = {}

matches_with_ratings = season_matches.with_columns(
    pl.struct(['winner_url', 'loser_url'])
        .map_elements(rate_players, return_dtype=pl.Struct)
        .alias('new_ratings')
).unnest('new_ratings')

In [75]:
rating = list(players_dict.values())[0]
rating.sigma

1.0723709081628914

In [76]:
player_ratings = pl.DataFrame({
    'url': [player_url for player_url in players_dict.keys()],
    'rating': [rating.mu for rating in players_dict.values()],
    'rating_95ci_lower': [rating.mu - 2*rating.sigma for rating in players_dict.values()],
    'rating_95ci_upper': [rating.mu + 2*rating.sigma for rating in players_dict.values()],
    'sigma': [rating.sigma for rating in players_dict.values()],
    'pi': [rating.pi for rating in players_dict.values()],
    'tau': [rating.tau for rating in players_dict.values()],
    'exposure': [rating.exposure for rating in players_dict.values()],
}).join(players, on='url').sort('rating', descending=True)

In [77]:
matches_with_difference = (
    matches_with_ratings
    .join(player_ratings.select('url', pl.col('rating').alias('winner_final_rating')), left_on='winner_url', right_on='url')
    .join(player_ratings.select('url', pl.col('rating').alias('loser_final_rating')), left_on='loser_url', right_on='url')
    .with_columns((pl.col('winner_final_rating') - pl.col('loser_final_rating')).alias('final_rating_difference'))
    .select('winner_url', 'winner', 'winner_final_rating', 'loser_url', 'loser', 'loser_final_rating', 'final_rating_difference')
)

# Match Analysis


If some player's rating is higher $\beta$ than another player's, the player may have about a 76% (specifically $\Phi(\frac {1}{\sqrt{2}}))$ chance to beat the other player. The default value of $\beta$ is $\frac{ 25 }{ 6 }$.

## Wins in order of rating difference

In [78]:
def best_wins(matches: pl.DataFrame, player_url: str) -> pl.DataFrame:
    return (
        matches
        .filter(pl.col('winner_url').str.contains(player_url))
        .sort('final_rating_difference')
        .select(pl.exclude(r'^.*url$'))
        .select(pl.exclude(r'^winner.*$'))
        .head(100)
    )

## Losses in order of rating difference

In [79]:
def worst_losses(matches: pl.DataFrame, player_url: str) -> pl.DataFrame:
    return (
        matches
        .filter(pl.col('loser_url').str.contains(player_url))
        .select(pl.exclude(r'^.*url$'))
        .select(pl.exclude(r'^loser.*$'))
        .sort('final_rating_difference')
    )

In [80]:
def compare_players(matches: pl.DataFrame, player1: str, player2: str) -> pl.DataFrame:
    output1 = widgets.Output()
    output2 = widgets.Output()
    output3 = widgets.Output()
    output4 = widgets.Output()

    with output1:
        display(best_wins(matches, player1))
    with output2:
        display(best_wins(matches, player2))
    with output3:
        display(worst_losses(matches, player1))
    with output4:
        display(worst_losses(matches, player2))

    win_columns = widgets.HBox([output1, output2])
    loss_columns = widgets.HBox([output3, output4])
    final_output = widgets.VBox([win_columns, loss_columns])

    display(final_output)

In [82]:
pl.Config(tbl_rows=100)
compare_players(matches_with_difference, candidate_urls[9], candidate_urls[10])